<a href="https://colab.research.google.com/github/yhfga18/ML_stuff/blob/main/7_RNN_LSTM_GRU_10_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch Tutorial - RNN & LSTM & GRU - Recurrent Neural Nets




In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameter
# MNIST Dataset consists of 28 x 28 images
# but this time, to use LSTM, we look at one row at a time, so input_size = 28. 
# also there are 28 columns so sequence_length is 28 too. 
input_size = 28
sequence_length = 28
hidden_size = 128 # arbitrary 
num_layers = 2 # stacking 2 RNNs together 

# other Hyper-parameter
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

In [ ]:
# load dataset 
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers # how many RNN layers wanted
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, # 28
                          hidden_size, # 128 arbitrary 
                          num_layers, # 2
                          batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes) # output = hidden, but specific to this project
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        # print(h0.size()) # [2, 100, 128]

        # out: batch_size, seq_length, hidden_size
        # out (100, 28, 128)
        out, _ = self.rnn(x, h0)
        
        # out (100, 128)
        out = out[:, -1, :]
        
        # out: (n, 10)
        out = self.fc(out)
        return out 

class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(GRU, self).__init__() # RNN → GRU
        self.num_layers = num_layers 
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, # rnn → gru, RNN → GRU
                          hidden_size, 
                          num_layers, 
                          batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes) 
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        # print(h0.size()) # [2, 100, 128]

        # out: batch_size, seq_length, hidden_size
        # out (100, 28, 128)
        out, _ = self.gru(x, h0) # rnn → gru
        
        # out (100, 128)
        out = out[:, -1, :]
        
        # out: (n, 10)
        out = self.fc(out)
        return out 

In [ ]:
model = GRU(input_size, hidden_size, num_layers, num_classes).to(device)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [ ]:
# Train the model (fit)
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # origin shape: [N, 1, 28, 28]
        # resized: [N, 28, 28]
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')


Epoch [1/2], Step [100/600], Loss: 0.5756
Epoch [1/2], Step [200/600], Loss: 0.3439
Epoch [1/2], Step [300/600], Loss: 0.2514
Epoch [1/2], Step [400/600], Loss: 0.2743
Epoch [1/2], Step [500/600], Loss: 0.2214
Epoch [1/2], Step [600/600], Loss: 0.0951
Epoch [2/2], Step [100/600], Loss: 0.1597
Epoch [2/2], Step [200/600], Loss: 0.1208
Epoch [2/2], Step [300/600], Loss: 0.0878
Epoch [2/2], Step [400/600], Loss: 0.1648
Epoch [2/2], Step [500/600], Loss: 0.1042
Epoch [2/2], Step [600/600], Loss: 0.1505


In [ ]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 test images: {acc} %')


Accuracy of the network on the 10000 test images: 97.54 %
